In [435]:
import keras
import numpy as np
from keras import Sequential
from sklearn.preprocessing import LabelEncoder #pour le one_hot_encoding.
from keras.layers import Dense, Flatten, TimeDistributed #Flatten pour mettre à plat une matrice.
from keras import Input, Model
from keras.layers import add, Activation #add pour sommer les couches
from keras.layers import Conv1D, AveragePooling1D #pour le pooling et la convolution.

# Prédiction de structure secondaire par réseau de convolution.

## 1) Processing des données

### a) Training_set

In [596]:
#extraction des seq
x_train = []

with open("../data/train.fasta", "r") as file:
    for line in file:
        x_train.append(list(line[:-1]))

In [597]:
#completion avec des "0" jusqu'a 759
for seq in x_train:
    while len(seq) < 760:
        seq.append("J")

In [598]:
print(len(x_train[1]))
print(len(x_train))

760
1348


In [599]:
#extraction des ddsp:
y_train = []

with open("../data/train.dssp", 'r') as file:
    for line in file:
        y_train.append(list(line[:-1]))

In [600]:
#completion jusqu'a 759:
for d in y_train:
    while len(d) < 760:
        d.append("*")

In [601]:
print(len(y_train[1]))
print(len(y_train))

760
1348


In [602]:
#traduction des X en one_hot_encoding:
all_one_hot_x_train = np.zeros((1348,760,21))

print(all_one_hot_x_train.shape)
classes = LabelEncoder()

for i in range(0,len(x_train)):    
    classes.fit(x_train[i])
    classes_i = classes.transform(x_train[i])
    one_hot_i = keras.utils.to_categorical(classes_i, num_classes= 21)  
    all_one_hot_x_train[i,:,:] = one_hot_i
print(all_one_hot_x_train.shape)

(1348, 760, 21)
(1348, 760, 21)


In [603]:
#traduction des Y en one_hot_encoding:
all_one_hot_y_train = np.zeros((1348,760,4))

print(all_one_hot_y_train.shape)
classes = LabelEncoder()

for j in range(0,len(y_train)):    
    classes.fit(y_train[j])
    classes_j = classes.transform(y_train[j])
    one_hot_j = keras.utils.to_categorical(classes_j, num_classes= 4)
    all_one_hot_y_train[j,:,:] = one_hot_j
print(all_one_hot_y_train.shape)

(1348, 760, 4)
(1348, 760, 4)


### b) Testing_set

In [604]:
#extraction des seq
x_test = []

with open("../data/blind.fasta", "r") as file:
    for line in file:
        x_test.append(list(line[:-1]))

In [605]:
#completion avec des "0" jusqu'a 759
for seq in x_test:
    while len(seq) < 760:
        seq.append("0")

In [606]:
y_test = []

with open("../data/blind.dssp", 'r') as file:
    for line in file:
        y_test.append(list(line[:-1]))

In [607]:
#completion jusqu'a 759:
for d in y_test:
    while len(d) < 760:
        d.append("0")

In [608]:
print(len(x_test))
print(len(y_test))

149
149


In [609]:
#traduction des X en one_hot_encoding:
all_one_hot_x_test = np.zeros((149,760,21))

print(all_one_hot_x_test.shape)
classes = LabelEncoder()

for i in range(0,len(x_test)):    
    classes.fit(x_test[i])
    classes_i = classes.transform(x_test[i])
    one_hot_i = keras.utils.to_categorical(classes_i, num_classes = 21)  
    all_one_hot_x_test[i,:,:] = one_hot_i
print(all_one_hot_x_test.shape)

(149, 760, 21)
(149, 760, 21)


In [610]:
#traduction des Y en one_hot_encoding:
all_one_hot_y_test = np.zeros((149,760,4))
print(all_one_hot_y_test.shape)
classes = LabelEncoder()

for j in range(0,len(y_test)):    
    classes.fit(y_test[j])
    classes_j = classes.transform(y_test[j])
    one_hot_j = keras.utils.to_categorical(classes_j, num_classes = 4)
    all_one_hot_y_test[j,:,:] = one_hot_j
print(all_one_hot_y_test.shape)

(149, 760, 4)
(149, 760, 4)


## 2) Construction du réseau CNN

In [611]:
model = Sequential()
model.add(Conv1D(filters=21, kernel_size=3, strides=1, padding= "same", activation = "relu"))
model.add(Conv1D(filters=21, kernel_size=3, strides=1, padding= "same", activation = "relu"))
#model.add(AveragePooling1D(pool_size = 3))
model.add(TimeDistributed(Dense(4,activation = "softmax")))

In [621]:
model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_80 (Conv1D)           (None, 760, 21)           1344      
_________________________________________________________________
conv1d_81 (Conv1D)           (None, 760, 21)           1344      
_________________________________________________________________
time_distributed_38 (TimeDis (None, 760, 4)            88        
Total params: 2,776
Trainable params: 2,776
Non-trainable params: 0
_________________________________________________________________


In [613]:
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=['accuracy'])

In [614]:
model.fit(all_one_hot_x_test,all_one_hot_y_test,epochs=100, batch_size = 30)

Epoch 1/100
149/149 [==============================] - 1s 8ms/step - loss: 1.3087 - accuracy: 0.6891
Epoch 2/100
149/149 [==============================] - 0s 1ms/step - loss: 1.1967 - accuracy: 0.8565
Epoch 3/100
149/149 [==============================] - 0s 1ms/step - loss: 1.1068 - accuracy: 0.8667
Epoch 4/100
149/149 [==============================] - 0s 1ms/step - loss: 1.0208 - accuracy: 0.8735
Epoch 5/100
149/149 [==============================] - 0s 1ms/step - loss: 0.9333 - accuracy: 0.8774
Epoch 6/100
149/149 [==============================] - 0s 1ms/step - loss: 0.8446 - accuracy: 0.8795
Epoch 7/100
149/149 [==============================] - 0s 1ms/step - loss: 0.7556 - accuracy: 0.8804
Epoch 8/100
149/149 [==============================] - 0s 1ms/step - loss: 0.6700 - accuracy: 0.8811
Epoch 9/100
149/149 [==============================] - 0s 1ms/step - loss: 0.5908 - accuracy: 0.8815
Epoch 10/100
149/149 [==============================] - 0s 1ms/step - loss: 0.5201 - accura

In [615]:
evaluation = model.evaluate(all_one_hot_x_test, all_one_hot_y_test)
print(evaluation)

149/149 [==============================] - 1s 6ms/step
[0.19379292508499735, 0.9050688743591309]


Attention cependant, ces performaances sont trop élevées, cela doit venir des 0 et des '*' que l'on a ajouté pour compléter.

In [616]:
predictions = model.predict(all_one_hot_x_test)

In [338]:
print(predictions[0,:,:])
print(predictions.shape)

[[6.4262701e-04 2.6572502e-01 3.6959460e-01 3.6403775e-01]
 [9.1757705e-05 2.7214795e-01 4.4501230e-01 2.8274798e-01]
 [5.9088008e-05 3.0065620e-01 3.8055032e-01 3.1873438e-01]
 ...
 [9.9998391e-01 1.0629631e-05 9.8438375e-07 4.5174593e-06]
 [9.9998391e-01 1.0629631e-05 9.8438375e-07 4.5174593e-06]
 [9.9948239e-01 2.2602608e-04 7.7003730e-05 2.1451579e-04]]
(149, 760, 4)


In [464]:
all_one_hot_y_test[0,:,3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [573]:
all_one_hot_y_test[0,:,3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

On a 4 classes (colonnes), avec 759 lignes (acides aminés).

In [620]:

predictions = model.predict(all_one_hot_x_test)

tp = 0
tn = 0
fn = 0
fp = 0
tot = 0
for i in range(len(predictions)):
    for j in range(len(predictions[i])):
        if all_one_hot_y_test[i,j,3] != 0.:
            predmax = -1
            predict_class = -1
            true_class = -1
            for k in range(len(predictions[i,j])):
                if predmax < predictions[i,j,k]:
                    predmax = predictions[i,j,k]
                    predict_class=k
                if all_one_hot_y_test[i,j,k] == 1.:
                    true_class = k
            if predict_class == true_class:
                tp = tp+1
            tot=tot+1
print("tp =", tp)
print("tot =", tot)
print("ACC:%5.3f"%(tp/tot*100))


tp = 1312
tot = 5257
ACC:24.957


5257
1312
ACC  0.249572


## 3) Réseau Resnet

In [ ]:
def residual_module(lay_i, n_filters):
    """
    Fonction de création de module.
    """
    save = lay_i #on stocke la première couche dans une variable.
    conv_1 = Conv1D(n_filters, (3), padding="same", activation="relu",
                    kernel_initializer="he_normal")(lay_i)
    conv_2 = Conv1D(n_filters, (3), padding="same", activation="linear",
                    kernel_initializer="he_normal")(conv_1)
    conc_1 = add([conv_2, save])
    output = Activation("relu")(conc_1)

    return output

def my_model7():
    n_residual = 4 #on chaine 4 fois les modules
    print("Simple residual network with {} modules".format(n_residual))
    inputs = Input(shape=(759, 21))
    residual_i = inputs
    for _ in range(n_residual):
        residual_i = residual_module(residual_i, 20)

    gavg_1 = AveragePooling1D((2), strides=(1))(residual_i)
    flat_1 = Flatten()(gavg_1)
    output = Dense(10, activation="softmax")(flat_1)

    model = Model(inputs=inputs, outputs=output)
    #Pour l'API fonctionnelle

#    plot_model(model, to_file="residual.png",
#               show_shapes=True, show_layer_names=True)
    print(model.summary())

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model
